In [1]:
# Set up an account access key
# spark.conf.set(
#  "fs.azure.account.key.<storage-account-name>.blob.core.windows.net",
#  "<storage-access-key>")

spark.conf.set(
  "fs.azure.account.key.databrickstore.blob.core.windows.net",
  "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg==")

NameError: name 'spark' is not defined

In [2]:
# dbutils.fs.ls("wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>")
dbutils.fs.ls("wasbs://dbdemo01@databrickstore.blob.core.windows.net")

NameError: name 'dbutils' is not defined

In [3]:
# mount a Blob storage container or a folder inside a container
# dbutils.fs.mount(
#   source = "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>",
#   mount_point = "<mount-point-path>",
#   extra_configs = <"<conf-key>": "<conf-value>">)
# [note] <mount_point> is a DBFS path and the path must be under /mnt

dbutils.fs.mount(
  source = "wasbs://dbdemo01@databrickstore.blob.core.windows.net",
  mount_point = "/mnt/dbdemo01",
  extra_configs = {"fs.azure.account.key.databrickstore.blob.core.windows.net": "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg=="})


NameError: name 'dbutils' is not defined

In [4]:
# Access files in your container as if they were local files
# (TEXT) df = spark.read.text("/mnt/%s/...." % <mount-point-path>)
# (JSON) df = spark.read.json("/mnt/%s/...." % <mount-point-path>)

df = spark.read.json( "/mnt/%s/small_radio_json.json" % "dbdemo01" )

# display(df)
df.show()

NameError: name 'spark' is not defined

In [5]:
# unmount
# dbutils.fs.unmount("<mount-point-path>")
# dbutils.fs.unmount("/mnt/dbdemo01")

In [6]:
specificColumnsDf = df.select("firstname", "lastname", "gender", "location", "level")
specificColumnsDf.show()

NameError: name 'df' is not defined

In [7]:
renamedColumnsDF = specificColumnsDf.withColumnRenamed("level", "subscription_type")
renamedColumnsDF.show()

NameError: name 'specificColumnsDf' is not defined

In [ ]:
# Apply some transformations to the data, then use the
# Data Source API to write the data back to another table in SQL DW.

# [note] the SQL date warehouse connector uses Azure Blob Storage as a temporary storage to upload data between Azure Databricks and Azure SQL Data Warehouse.

## SQL Data Warehouse related settings
dwTable= "mytable001"
dwDatabase = "sqldwdemo001"
dwServer = "sqldwdemoserver001" 
dwUser = "yoichika"
dwPass = "P@ssw0rd____"
dwJdbcPort =  "1433"
dwJdbcExtraOptions = "encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"
sqlDwUrl = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass + ";$dwJdbcExtraOptions"
sqlDwUrlSmall = "jdbc:sqlserver://" + dwServer + ".database.windows.net:" + dwJdbcPort + ";database=" + dwDatabase + ";user=" + dwUser+";password=" + dwPass


tempDir = "wasbs://dbdemo01tmp@databrickstore.blob.core.windows.net/tempDirs"

#sc._jsc.hadoopConfiguration().set(
#  "fs.azure.account.key.<your-storage-account-name>.blob.core.windows.net",
#  "<your-storage-account-access-key>")
acntInfo = "fs.azure.account.key.databrickstore.blob.core.windows.net"
sc._jsc.hadoopConfiguration().set(
  acntInfo, 
  "S1PtMWvUw5If1Z8FMzXAxC7OMw9G5Go8BGCXJ81qpFVYpZ9dpXOnU4zlg0PbldKkbLIbmbv02WoJsgYLGKIfgg==")

## Loading transformed dataframe (renamedColumnsDF) into SQLDW
spark.conf.set("spark.sql.parquet.writeLegacyFormat","true")

## This snippet creates a table called 'dwTable' in the SQL database.
#df.write \
#  .format("com.databricks.spark.sqldw") \
#  .option("url", "jdbc:sqlserver://<the-rest-of-the-connection-string>") \
#  .option("forward_spark_azure_storage_credentials", "true") \
#  .option("dbtable", "my_table_in_dw_copy") \
#  .option("tempdir", "wasbs://<your-container-name>@<your-storage-account-name>.blob.core.windows.net/<your-directory-name>") \
#  .save()

renamedColumnsDF.write \
  .format("com.databricks.spark.sqldw") \
  .option("url", sqlDwUrlSmall) \
  .option("dbtable", dwTable) \
  .option( "forward_spark_azure_storage_credentials","true") \
  .option("tempdir", tempDir) \
  .mode("overwrite") \
  .save()